# Previsão para linhas de ônibus na cidade do Rio de Janeiro

<b>Aluno: Victor Ribeiro </b>
<br>
<b>DRE: 120035521</b>

O objetivo deste trabalho é apresentar uma solução em Python que seja capaz de prever:
 - A posição de um ônibus a partir dado um horário
 - O horário a partir da posição do um ônibus

### 1 - Preparação dos dados

Os dados de GPS dos ônibus usados para criar o modelo vieram de arquivos JSON, dividos em intervalos de 1 hora, do dia 25/04/2024 à 10/05/2024. 
Os arquivos de teste usados para fazer a previsão constam do dia 16/05/2024 à 20/05/2024.
Inicialmente, foi necessário carregar os dados do GPS em um banco de dados PostgreSQL usando a extensão PostGIS. Para isso, alguns filtros foram usados para selecionar apenas os dados úteis, diminuindo o tempo de interação com o banco e o tamanho em disco:
- Ao analisar os arquivos de teste, selecionei apenas a linhas que seriam usadas na previsão. Dessa forma, apenas 48 linhas de ônibus do conjunto de arquivos de treino foram utilizadas. As demais, não foram carregadas ao banco.
- Para carregar o banco, transformei os arquivos JSON em planilhas CSV e usei o método COPY do PostgreSQL, diminuindo consideravelmente o tempo de inserção.
- Ao criar a tabela, dividi a mesma em partições por dias, facilitando a execução das consultas na tabela.

### 2 - Identificação das garagens de ônibus

Com o objetivo criar os trajetos de cada linha de ônibus, foi necessário primeiro detectar os ônibus que estavam parados na garagem. Essa identificação permitiu filtrar os dados, removendo os pontos onde os ônibus estavam na garagem ou em trajeto de ida/volta.<br><br>
Para encontrar estes pontos, utilizei apenas os dados onde os ônibus estavam parados (velocidade = 0) e no horário da madrugada (entre 23h-4h). Após coletar estes dados do banco de dados, utilizei o DBScan para criar clusters com um raio de 100 metros. Como resultado, encontrei 21 clusters que eram referentes à garagem de ônibus. Realizei uma dupla checagem, conferindo GoogleMaps/OpenStreetMap e verificando que estes pontos realmente eram garagens. <br>
Obtive o ponto central de cada cluster e inclui em um tabela no banco de dados.


### 3 - Identificação dos pontos finais e iniciais para cada linha de ônibus